# MTH9875 Homework 11
### ShengQuan Zhou

Assigned: November 8, 2018.
Due: November 22, 2017 (yes, Thanksgiving Day!) by 6pm. 

Late homework **will not be accepted**.

<font color = "red">Homework is to be done by each student individually.  To receive full credit, you must email a completed copy of this iPython notebook to Victor Istratov (stavka@gmail.com), Yassine Ghalem (yassine.ghalem@gmail.com) and Jim Gatheral (jim.gatheral@gmail.com) by the due date and time.  All R-code must run correctly and solutions must be written up neatly in Markdown/LaTeX format.

### 0. (0 points)

Read Chapters 9 and 10 of The Volatility Surface

### Answer:

Done.

## Comparing stochastic and local volatility pricing of lookback options

$$
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\E}{\mathbb{E} }
\def\Cov{{ \mbox{Cov} }}
\def\Var{{ \mbox{Var} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\PP}{\mathbb{P} }
%\newcommand{\Pr}{\mathrm{Pr} }
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\RR}{\mathbb{R} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\sgn}{\mathrm{sgn} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
%\newcommand{\HS}{\mathrm{HS} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
%\newcommand{\LL}{\mathrm{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\p}{\partial}
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bi}{\begin{itemize}}
\newcommand{\ei}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\cO}{\mathcal{O}}
\newcommand{\cF}{\mathcal{F}}
\newcommand{\cL}{\mathcal{L}}
$$



In the following, a lookback call is defined to pay

$$
\left(S_{\text{max}}-K\right)^+
$$
where $S_{\text{max}}$ is the maximum and $K$ is the strike price.

### 1. (8 points)

Use (for example) the stochastic volatility and local volatility Monte Carlo codes (`HestonMCxmax` and `HestonLvMCxmax` respectively) from Lecture 11 to price a one year at-the-money lookback call
that is monitored once a day (assume 252 working days in a year).

Use the usual HN parameters  - namely:

$$
\begin{eqnarray*}
v&=& 0.04\\ \bar v &=& 0.04\\ \lambda&=&10\\ \eta&=&1\\\rho&=&-1.
\end{eqnarray*}
$$

Use 252 time steps and 20,000 paths.  Do not use Richardson extrapolation.

### Solution:

In [36]:
paramsCh4 <- list(lambda = 10,rho = -1,eta = 1,vbar = 0.04,v = 0.04)

In [37]:
evolveAlfonsiF <- function(v,x,dt,W1,W2,L){
  
  eldt2 <- exp(-lambda*dt/2);
  
  #Variance process 
  vbarp <- vbar - eta^2/(4*lambda);
  psi <- (1-eldt2)/lambda;
  v1 <- v*eldt2+lambda*vbarp*psi;
  v2 <- (v1 > 0) * v1; # Take v2 = 0 if v1<0, else v2=v1
  par <- sqrt(v2) + eta/2 * sqrt(dt)*W2;
  vf <- eldt2*par*par +lambda*vbarp*psi + v1 - v2;
  # Full truncation
  
  # Log-stock process (Andersen equation (33))
  vvf <- (v+vf > 0) * (v+vf);
  dw <- vvf/2*dt;
  x <- x  - dw/2 + rho2m1*sqrt(dw)*W1 +
    rho/eta*(lambda*dw + vf-v -lambda*vbar*dt) ;
  # Impose martingale constraint
  x <- x - log(mean(exp(x)));
  v <- vf;
  return(cbind(x,v));
}

In [38]:
HestonMCxmax <- function(params){

  is.even <- function(j){as.logical((j+1) %% 2)}
  
  res <- function(S0, T, AK, N, m, evolve,exactVols=NULL)
  {
    
    lambda <<- params$lambda 
    rho <<- params$rho 
    eta <<- params$eta 
    vbar <<- params$vbar 
    v0 <<- params$v 
    
    n <- m*2  #n is number of timesteps = 2*m so we can use Romberg extrapolation
    sqrt2 <- sqrt(2) 
    rho2m1 <<- sqrt(1-rho*rho) 
    vbarp <<- vbar - eta^2/(4*lambda) 
    
    negCount <- 0 
    
    # We use a vertical array, one element per M.C. path
    x <- numeric(N);  v <- rep(1,N)*v0  
    xmax <- x 
    xm <- x  
    vm <- v
    xmaxm <- xmax 
    xmin <- numeric(N)  
    xminm <- numeric(N) 
    W1m <- numeric(N) 
    W2m <- numeric(N)  
    
    # Loop for bias computation (N small, n big)
    for (i in 1:n)
    {
      # Two sets of correlated normal random vars.
      
      W1 <- rnorm(N) 
      W2 <- rnorm(N) 
      W1 <- W1 - mean(W1)
      W1 <- W1/sd(W1) 
      W2 <- W2 - mean(W2)
      W2 <- W2/sd(W2) 
      # Now W1 and W2 are forced to have mean=0 and sd=1
      
      W2p <- W2 - cor(W1,W2)*W1  # Eliminate actual correlation
      W2p <- W2p - mean(W2p)  
      W2 <- W2p/sd(W2p)  
      # Now W1 and W2 have mean=0, sd=1 and correlation=0
      
      L <- rbinom(N, size=1, prob=1/2)  # Bernoulli rv for NV step
      
      # Add code for subgrid
      W1m <- W1m + W1/sqrt2
      W2m <- W2m + W2/sqrt2  # N(0,1) rv's for subgrid
      
      if (is.even(i)) {
        #print(c(i,mean(W1m),mean(W2m),sd(W1m),sd(W2m),cor(W1m,W2m))) 
        resm <- evolve(vm,xm,T/m,W1m,W2m,L) 
        xnewm <- resm[,1] 
        xmaxm <- (xnewm>xmaxm)*(xnewm-xmaxm)+xmaxm 
        xminm <- (xnewm<xminm)*(xnewm-xminm)+xminm 
        xm <- xnewm 
        vm <- resm[,2] 
        W1m <- numeric(N)  
        W2m <- numeric(N) 
      }
      
      res <- evolve(v,x,T/n,W1,W2,L) 
      xnew <- res[,1] 
      xmax <- (xnew>xmax)*(xnew-xmax)+xmax 
      xmin <- (xnew<xmin)*(xnew-xmin)+xmin 
      x <- xnew 
      v <- res[,2] 
      
    }
    
    return(data.frame(x,xm,v,vm,xmax,xmaxm,xmin,xminm)) 
    
  }
  return(res) 
}

In [39]:
# Monte Carlo with Richardson extrapolation
HestonLvMCxmax <- function(params){

  is.even <- function(j){as.logical((j+1) %% 2)}
  
  res <- function(S0, T, N, m)
  {
    
    lambda <- params$lambda;
    rho <- params$rho;
    eta <- params$eta;
    vbar <- params$vbar;
    v0 <- params$v;
    
    lambdap <- lambda+eta/2;
    vbarp <- vbar*lambda/lambdap;
    
    n <- m*2;
    dt <- T/n;
    dtm <- dt*2;
    
    sqrtdt <- sqrt(dt);
    sqrtdtm <- sqrt(dtm);
    sqrt2 <- sqrt(2);
    
    # We use a vertical array, one element per M.C. path
    x <- numeric(N); vloc <- rep(v0,N); 
    xm <- numeric(N); vlocm <- rep(v0,N); 
    xmax <- numeric(N); xmaxm <- numeric(N);
    xmin <- numeric(N); xminm <- numeric(N);
    W1m <- numeric(N);
    
    # Loop over timesteps
    for (i in 1:n)
    {
      W1 <- rnorm(N);
      W1 <- W1 - mean(W1); W1 <- W1/sd(W1);
      # Now W1 is forced to have mean=0 and sd=1  
      
      # Add code for subgrid
      W1m <- W1m + W1/sqrt2; # N(0,1) rv's for subgrid
      
      # Compute local variance from formula 4.1 of TVS
      ti <- i*dt;
      elti <- exp(-lambdap*ti);	
      vlocf <- (v0-vbarp)*elti+vbarp-eta*x*(1-elti)/(lambdap*ti);
      vlocf <- (vlocf>=0)*vlocf;
      
      # Approximate integrated variance
      w <- (vlocf +vloc)/2*dt;
      
      xnew <- x - w/2 + sqrt(w)*W1;
      xmax <- (xnew>xmax)*(xnew-xmax)+xmax;
      xmin <- (xnew<xmin)*(xnew-xmin)+xmin;
      x <- xnew;
      vloc <- vlocf;
      
      
      if (is.even(i)) {
        
        vlocfm <- (v0-vbarp)*elti+vbarp-eta*xm*(1-elti)/(lambdap*ti);
        vlocfm <- (vlocfm>=0)*vlocfm;
        
        wm <- (vlocfm+vlocm)/2*dtm;
        xnewm <- xm - wm/2 + sqrt(wm)*W1m;
        xmaxm <- (xnewm>xmaxm)*(xnewm-xmaxm)+xmaxm;
        xminm <- (xnewm<xminm)*(xnewm-xminm)+xminm;
        xm <- xnewm;
        vlocm <- vlocf;
        W1m <- numeric(N);
      } 
      
      
    }
    
    return(data.frame(x,xm,vloc,vlocm,xmax,xmaxm,xmin, xminm));
    
  }
  return(res);
}

In [ ]:
sampleLV <- HestonLvMCxmax(paramsCh4)(S0=1, T=1, N=20000, m=252/2);
sampleSV <- HestonMCxmax(paramsCh4)(S0=1, T=1, N=20000, m=252/2,evolve=evolveAlfonsiF);

Price <- function(data,strike){
  Smax <- exp(data$xmax)
  K <- strike
  V <- (Smax>K)*(Smax-K)
  AV <- mean(V)
  return(AV)  
}

atmSV<-Price(sampleSV,strike=1)
atmLV<-Price(sampleLV,strike=1)
cat("Stochastic Vol price: ",atmSV,"\n")
cat("Local Vol price: ",atmLV)

### 2. (4 points) 

Plot local volatility and stochastic volatility prices of lookback options for a range of strikes from $1.0$ to $1.4$, reproducing Figure 8 of Lecture 11.

### Solution:

In [ ]:
# Define lookback option based on the payoff
LookBack <- function(data,strike)
{
    S_max <- exp(data$xmax)
    K <- strike
    V <- (S_max>K)*(S_max-K)
    AV <- mean(V)
    return(AV)  
}

In [ ]:
strikes <- seq(1,1.4,0.02)

resSV <- sapply(strikes,function(K){LookBack(sampleSV,strike=K)})
resLV <- sapply(strikes,function(K){LookBack(sampleLV,strike=K)})

plot(strikes, resSV, col="blue",type="l",ylab="Price of Lookback Call Option",xlab="Strike",ylim=c(0,0.15))
points(strikes, resLV, col="green",type="l")
legend("topright",c("stochastic volatility","local volatility"),col=c("blue","green"),lty=c(1,1))

### 3. (6 points) 

What is the intuition behind the difference in price under stochastic volatility and local volatility assumptions respectively?  Appeal to the dynamics of the implied volatility skew in
your answer. 

### Answer:

When we sell a lookback call option struck at $K \gt S$, we hedge it by buying two European call options struck at $K$. If the stock price reaches $K$, we need to sell two European calls struck at $K$, and buy two new calls struck at $K+\Delta K$, which is equivalent to $2C(K+\Delta K,K)-2C(K+\Delta K,K+\Delta K)$, proportional to the negative skew. 
Under local volatility, the skew is flatter, we will get less at each time we rebalance the hedging portfolio. So we charge more under local volatility assumptions. Thus, the prices under stochastic volatility are lower then those under local volatility.

## The effect of discreteness

### 4. (6 points) 

Repeat Problem 1 with weekly monitoring (assume 52 weeks).  Estimate the difference in price using the Broadie, Glasserman and Kou adjustment.  Compare with your Monte Carlo results and comment.

### Solution:

In [ ]:
sampleLV_Weekly <- HestonLvMCxmax(paramsCh4)(S0=1, T=1, N=20000, m=52/2);
sampleSV_Weekly <- HestonMCxmax(paramsCh4)(S0=1, T=1, N=20000, m=52/2,evolve=evolveAlfonsiF);

atmLV_Weekly <- LookBack(sampleLV_Weekly,strike=1)
atmSV_Weekly <- LookBack(sampleSV_Weekly,strike=1)
cat("Weekly monitoring, Stochastic Vol price: ",atmSV_Weekly," Local Vol price: ",atmLV_Weekly)

Paste the code for Heston model.

In [ ]:
callOption <- function(phi, k, t)
{
    integrand <-  function(u){Re(exp(-1i*u*k)*phi(u - 1i/2, t)/(u^2 + 1/4))};
    res <- 1 - exp(k/2)/pi*integrate(integrand,lower=0,upper=Inf,rel.tol=0.0000000001,subdivisions=1000)$value;
    return(res);
}

In [ ]:
BSFormula <- function(S0, K, T, r, sigma)
{
    x <- log(S0/K)+r*T;
    sig <- sigma*sqrt(T);
    d1 <- x/sig+sig/2;
    d2 <- d1 - sig;
    pv <- exp(-r*T);
    return( S0*pnorm(d1) - pv*K*pnorm(d2));
}

In [ ]:
BSImpliedVolCall <- function(S0, K, T, r, C)
{
    nK <- length(K);
    sigmaL <- rep(1e-10,nK);
    CL <- BSFormula(S0, K, T, r, sigmaL);
    sigmaH <- rep(10,nK);
    CH <- BSFormula(S0, K, T, r, sigmaH);
    while (mean(sigmaH - sigmaL) > 1e-10)
    {
        sigma <- (sigmaL + sigmaH)/2;
        CM <- BSFormula(S0, K, T, r, sigma);
        CL <- CL + (CM < C)*(CM-CL);
        sigmaL <- sigmaL + (CM < C)*(sigma-sigmaL);
        CH <- CH + (CM >= C)*(CM-CH);
        sigmaH <- sigmaH + (CM >= C)*(sigma-sigmaH);
    }
    return(sigma);
}

In [ ]:
bsvol <- function(phi,k, t)
{
    BSImpliedVolCall(1, exp(k), t, 0, callOption(phi,k,t))
}

In [ ]:
phiHeston <- function(params)
{
    lambda <- params$lambda;
    rho <- params$rho;
    eta <- params$eta;
    vbar <- params$vbar;
    v <- params$v;
    
    function(u, t)
    {
        al <- -u*u/2 - 1i*u/2;
        bet <- lambda - rho*eta*1i*u;
        gam <- eta^2/2;
        d <- sqrt(bet*bet - 4*al*gam);
        rp <- (bet + d)/(2*gam);
        rm <- (bet - d)/(2*gam);
        g <- rm / rp;
        D <- rm * (1 - exp(-d*t))/ (1 - g*exp(-d*t));
        C <- lambda * (rm * t - 2/eta^2 * log( (1 - g*exp(-(d*t)))/(1 - g) ) );
        return(exp(C*vbar + D*v));
    }
}

In [ ]:
impvolHeston <- function(params)
{
    function(k, t)
    {
        bsvol(phiHeston(params),k,t)
    }
}

In [ ]:
sigma <- impvolHeston(paramsCh4)(k=0,t=1)
cat("atm implied vol: ", sigma)

In [ ]:
LookBackCtn <- function(strike)
{
    return(2*BSFormula(S0=1, K=strike, T=1, r=0, sigma=sigma))
}

In [ ]:
beta<-0.5826

lb_Daily <- LookBackCtn(exp(-beta*sigma*sqrt(1/252)))-(1-exp(-beta*sigma*sqrt(1/252)))
lb_Weekly <- LookBackCtn(exp(-beta*sigma*sqrt(1/52)))-(1-exp(-beta*sigma*sqrt(1/52)))

In [ ]:
x<-c("Stochastic Vol",atmSV,atmSV_Weekly,atmSV-atmSV_Weekly)
y<-c("Local Vol",atmLV,atmLV_Weekly,atmLV-atmLV_Weekly)
z<-c("Broadie, Glasserman and Kou adjustment",lb_Daily,lb_Weekly,lb_Daily-lb_Weekly)
data = rbind(x,y,z)
colnames(data) = c('model', 'daily','weekly','diff daily-weekly')
rownames(data) = c()
data

Note: Using BGK adjustment, the price difference between daily and weekly monitoring is one order of magnitude smaller than that from Monte Carlo, under stochastic volatility or local volatility.